# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-history4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Civilization explorative search") 

Consider the following exploratory information need:

> Investigate the Ancient Egypt and the related archeological explorations and findings.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q13444093`| Meidum Pyramid    | node |
| `wd:Q41127`| Poseidon   | node |





Also consider

```
wd:Q13444093 ?p ?obj .
```

is the BGP to retrieve all **properties of Meidum Pyramid**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for Ancient Egypt

2. Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

3. Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

4. Identify the BGP for archeologist.

5. For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

6. Find the BGP for deity.

7. List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

8. Compare ancient Greece with ancient Egypt and find how many deities each one has.


In [2]:
## startup the evaluation
# setup the file and create the empty json
ipname = "h4.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 1_4


## Task 1

In [3]:
# find the properties of Meidum Pyramid
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q13444093 ?prop ?obj.
    ?prop sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P1481'), ('name', 'vici.org ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1566'), ('name', 'GeoNames ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material')]
[('prop', 'http://www.wikidata.org/prop/direct/P2044'), ('name', 'elevation above sea level')]
[('prop', 'http://www.wikidata.org/prop/direct/P2048'), ('name', 'height')]
[('prop', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2326'), ('name', 'GNS Unique

In [4]:
# use the property commissioned by
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q13444093 wdt:P88 ?obj.
    ?obj sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q189371'), ('name', 'Sneferu')]
[('obj', 'http://www.wikidata.org/entity/Q220240'), ('name', 'Huni')]
2


In [5]:
# find the properties of the commissioner of the Meidum Pyramid
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q13444093 wdt:P88 ?obj.
    ?obj ?prop ?x.
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('prop', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1309'), ('name', 'EGAXA ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('prop', 'http://www.wikidata.org/prop/direct/P1366'), ('name', 'replaced by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1711'), ('name', 'British Museum person or institution ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/

In [6]:
# look for the object of noble title
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q13444093 wdt:P88 ?obj.
    ?obj wdt:P97 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q37110'), ('name', 'pharaoh')]
1


In [7]:
# use the nationality of the commissioners
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q13444093 wdt:P88 ?obj.
    ?obj wdt:P27 ?x.
    ?x sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt')]
1


In [8]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q11768"
og_name = "Ancient Egypt"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_4.json
JSON object updated


`wd:Q11768` -> Ancient Egypt

## Task 2

In [10]:
# list the pharaoh's properties
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?phar ?prop wd:Q37110.
    ?prop sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('prop', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P2868'), ('name', 'subject has role')]
[('prop', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('prop', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('prop', 'http://www.wikidata.org/prop/direct/P39'), ('name', 'position held')]
[('prop', 'http://www.wikidata.org/prop/direct/P460'), ('name', 'said to be the same as')]
[('pro

In [56]:
# I have to find the children and date of birth
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?phar wdt:P39 wd:Q37110;
        ?prop ?x.
    ?prop sc:name ?name.
    FILTER(REGEX(?name,"[C,c]hild") || REGEX(?name,"[D,d]ate")).  
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
[('prop', 'http://www.wikidata.org/prop/direct/P570'), ('name', 'date of death')]
[('prop', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child')]
3


In [11]:
# filter for pharaohs born between -1500 and -1000 and find who had more than 8 children
queryString = """
SELECT ?phar ?name COUNT(DISTINCT ?ch) AS ?children
WHERE { 
    ?phar wdt:P39 wd:Q37110;
        wdt:P569 ?x;
        wdt:P40 ?ch.
    FILTER(?x>xsd:datetime("-1500-01-01") && ?x<xsd:datetime("-1000-01-01") ).  
    ?phar sc:name ?name.
}
GROUP BY ?phar ?name
HAVING (COUNT(DISTINCT ?ch) > 8)
"""

print("Results")
x=run_query(queryString)

Results
[('phar', 'http://www.wikidata.org/entity/Q42606'), ('name', 'Amenhotep III'), ('children', '10')]
[('phar', 'http://www.wikidata.org/entity/Q152230'), ('name', 'Tiye'), ('children', '9')]
[('phar', 'http://www.wikidata.org/entity/Q1523'), ('name', 'Ramesses II'), ('children', '18')]
[('phar', 'http://www.wikidata.org/entity/Q1528'), ('name', 'Ramesses III'), ('children', '10')]
[('phar', 'http://www.wikidata.org/entity/Q81794'), ('name', 'Akhenaten'), ('children', '9')]
5


In [12]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    children = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= children
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_4.json
JSON object updated


## Task 3

In [13]:
# look for the instance of Ancient Egypt
queryString = """
SELECT DISTINCT ?obj ?o
WHERE { 
    wd:Q11768 wdt:P31 ?obj.
    ?obj sc:name ?o.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3024240'), ('o', 'historical country')]
[('obj', 'http://www.wikidata.org/entity/Q3502482'), ('o', 'cultural region')]
[('obj', 'http://www.wikidata.org/entity/Q1620908'), ('o', 'historical region')]
[('obj', 'http://www.wikidata.org/entity/Q28171280'), ('o', 'ancient civilization')]
4


In [14]:
# find properties of object that are instances of ancient civilization
queryString = """
SELECT DISTINCT ?prop ?o
WHERE { 
    ?obj wdt:P31 wd:Q28171280;
        ?prop ?x.
    ?prop sc:name ?o.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1014'), ('o', 'Art & Architecture Thesaurus ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1036'), ('o', 'Dewey Decimal Classification')]
[('prop', 'http://www.wikidata.org/prop/direct/P1082'), ('o', 'population')]
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('o', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1151'), ('o', "topic's main Wikimedia portal")]
[('prop', 'http://www.wikidata.org/prop/direct/P122'), ('o', 'basic form of government')]
[('prop', 'http://www.wikidata.org/prop/direct/P1225'), ('o', 'U.S. National Archives Identifier')]
[('prop', 'http://www.wikidata.org/prop/direct/P1245'), ('o', 'OmegaWiki Defined Meaning')]
[('prop', 'http://www.wikidata.org/prop/direct/P1269'), ('o', 'facet of')]
[('prop', 'http://www.wikidata.org/prop/direct/P1282'), ('o', 'OpenStreetMap tag or key')]
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('o', 'Gran Enciclopèdia Catalana ID')]
[('pro

In [17]:
# find the ancient civilizations possibily with their duration
queryString = """
SELECT DISTINCT ?obj ?o ?duration
WHERE { 
    ?obj wdt:P31 wd:Q28171280.
    OPTIONAL{ 
        ?obj wdt:P580 ?start;
            wdt:P582 ?end.
        BIND(year(?end)-year(?start) AS ?duration). 
    }
    ?obj sc:name ?o.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11768'), ('o', 'Ancient Egypt'), ('duration', '4395')]
[('obj', 'http://www.wikidata.org/entity/Q11772'), ('o', 'Ancient Greece'), ('duration', '1800')]
[('obj', 'http://www.wikidata.org/entity/Q134178'), ('o', 'Minoan civilization'), ('duration', '901')]
[('obj', 'http://www.wikidata.org/entity/Q134178'), ('o', 'Minoan civilization'), ('duration', '2200')]
[('obj', 'http://www.wikidata.org/entity/Q42534'), ('o', 'Indus Valley Civilization'), ('duration', '1200')]
[('obj', 'http://www.wikidata.org/entity/Q1551054'), ('o', 'Israelite period'), ('duration', '614')]
[('obj', 'http://www.wikidata.org/entity/Q41741'), ('o', 'Thrace')]
[('obj', 'http://www.wikidata.org/entity/Q128904'), ('o', 'Elam')]
[('obj', 'http://www.wikidata.org/entity/Q185068'), ('o', 'Urartu')]
[('obj', 'http://www.wikidata.org/entity/Q11767'), ('o', 'Mesopotamia')]
[('obj', 'http://www.wikidata.org/entity/Q32047'), ('o', 'Illyria')]
[('obj', 'http://www.wikidata.org/

In [20]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    duration = 0
    if len(i) == 3:
        duration = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= duration
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_4.json
JSON object updated


## Task 4

In [21]:
# find how object are related to instances of ancient civilization
queryString = """
SELECT DISTINCT ?prop ?o
WHERE { 
    ?obj wdt:P31 wd:Q28171280.
    ?x ?prop ?obj.
    ?prop sc:name ?o.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1001'), ('o', 'applies to jurisdiction')]
[('prop', 'http://www.wikidata.org/prop/direct/P101'), ('o', 'field of work')]
[('prop', 'http://www.wikidata.org/prop/direct/P1049'), ('o', 'worshipped by')]
[('prop', 'http://www.wikidata.org/prop/direct/P112'), ('o', 'founded by')]
[('prop', 'http://www.wikidata.org/prop/direct/P119'), ('o', 'place of burial')]
[('prop', 'http://www.wikidata.org/prop/direct/P1269'), ('o', 'facet of')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('o', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P135'), ('o', 'movement')]
[('prop', 'http://www.wikidata.org/prop/direct/P136'), ('o', 'genre')]
[('prop', 'http://www.wikidata.org/prop/direct/P1365'), ('o', 'replaces')]
[('prop', 'http://www.wikidata.org/prop/direct/P1376'), ('o', 'capital of')]
[('prop', 'http://www.wikidata.org/prop/direct/P138'), ('o', 'named after')]
[('prop', 'http://www.w

In [22]:
# use field of work
queryString = """
SELECT DISTINCT ?x ?o
WHERE { 
    ?obj wdt:P31 wd:Q28171280.
    ?x wdt:P101 ?obj.
    ?x sc:name ?o.
} 
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q17122314'), ('o', 'Greek Study Association')]
[('x', 'http://www.wikidata.org/entity/Q2732854'), ('o', 'Epigraphical Museum')]
[('x', 'http://www.wikidata.org/entity/Q499357'), ('o', 'Andreas Luther')]
[('x', 'http://www.wikidata.org/entity/Q1985476'), ('o', 'Charles Thomas Newton')]
[('x', 'http://www.wikidata.org/entity/Q4446512'), ('o', 'Igor Surikov')]
[('x', 'http://www.wikidata.org/entity/Q2342814'), ('o', 'Alexandre Bertrand')]
[('x', 'http://www.wikidata.org/entity/Q2976497'), ('o', 'Classical Association')]
[('x', 'http://www.wikidata.org/entity/Q83196530'), ('o', 'Markus Eberl')]
[('x', 'http://www.wikidata.org/entity/Q64563606'), ('o', "Vincent Robin d'Arba Desborough")]
[('x', 'http://www.wikidata.org/entity/Q1523942'), ('o', 'Gilbert Charles-Picard')]
10


In [23]:
# list the occupations 
queryString = """
SELECT DISTINCT ?occ ?o
WHERE { 
    ?obj wdt:P31 wd:Q28171280.
    ?x wdt:P101 ?obj;
        wdt:P106 ?occ.
    ?occ sc:name ?o.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q26425137'), ('o', 'mykenologist')]
[('occ', 'http://www.wikidata.org/entity/Q121594'), ('o', 'professor')]
[('occ', 'http://www.wikidata.org/entity/Q13418253'), ('o', 'philologist')]
[('occ', 'http://www.wikidata.org/entity/Q14467526'), ('o', 'linguist')]
[('occ', 'http://www.wikidata.org/entity/Q1622272'), ('o', 'university teacher')]
[('occ', 'http://www.wikidata.org/entity/Q1792450'), ('o', 'art historian')]
[('occ', 'http://www.wikidata.org/entity/Q201788'), ('o', 'historian')]
[('occ', 'http://www.wikidata.org/entity/Q333634'), ('o', 'translator')]
[('occ', 'http://www.wikidata.org/entity/Q36180'), ('o', 'writer')]
[('occ', 'http://www.wikidata.org/entity/Q37226'), ('o', 'teacher')]
[('occ', 'http://www.wikidata.org/entity/Q901'), ('o', 'scientist')]
[('occ', 'http://www.wikidata.org/entity/Q1350189'), ('o', 'egyptologist')]
[('occ', 'http://www.wikidata.org/entity/Q15632632'), ('o', 'epigrapher')]
[('occ', 'http://www.wikidata.org

In [25]:
# list the occupations 
queryString = """
SELECT wd:Q3621491 AS ?occ ?o
WHERE { 
   wd:Q3621491  sc:name ?o.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('occ', 'http://www.wikidata.org/entity/Q3621491'), ('o', 'archaeologist')]
1


In [26]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q3621491"
og_name = "archaeologist"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_4.json
JSON object updated


`Q3621491` -> archeologist

## Task 5

In [28]:
# find the number of archologists for each ancient civilization
queryString = """
SELECT ?obj ?o COUNT(DISTINCT ?x) AS ?archeologists
WHERE { 
    ?obj wdt:P31 wd:Q28171280.
    OPTIONAL{?x wdt:P101 ?obj;
        wdt:P106/wdt:P279* wd:Q3621491.}
    ?obj sc:name ?o.
} 
GROUP BY ?obj ?o
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q99304188'), ('o', 'Thracian'), ('archeologists', '0')]
[('obj', 'http://www.wikidata.org/entity/Q11767'), ('o', 'Mesopotamia'), ('archeologists', '1')]
[('obj', 'http://www.wikidata.org/entity/Q35355'), ('o', 'Sumer'), ('archeologists', '0')]
[('obj', 'http://www.wikidata.org/entity/Q25558247'), ('o', 'Chinese civilization'), ('archeologists', '0')]
[('obj', 'http://www.wikidata.org/entity/Q2429397'), ('o', 'Ancient Carthage'), ('archeologists', '7')]
[('obj', 'http://www.wikidata.org/entity/Q173082'), ('o', 'Dacia'), ('archeologists', '0')]
[('obj', 'http://www.wikidata.org/entity/Q32047'), ('o', 'Illyria'), ('archeologists', '0')]
[('obj', 'http://www.wikidata.org/entity/Q28405'), ('o', 'Gojoseon'), ('archeologists', '0')]
[('obj', 'http://www.wikidata.org/entity/Q38060'), ('o', 'Gaul'), ('archeologists', '3')]
[('obj', 'http://www.wikidata.org/entity/Q8735'), ('o', 'Medes'), ('archeologists', '0')]
[('obj', 'http://www.wikidata.org/e

In [29]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    arch = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= arch
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 1_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_4.json
JSON object updated


## Task 6

In [31]:
# find the class of Poseidon
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q41127 wdt:P31 ?obj.
    ?obj sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1916821'), ('name', 'water deity')]
[('obj', 'http://www.wikidata.org/entity/Q22989102'), ('name', 'Greek deity')]
2


In [32]:
# find the superclass of Greek deity
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q22989102 wdt:P279 ?obj.
    ?obj sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q178885'), ('name', 'deity')]
[('obj', 'http://www.wikidata.org/entity/Q22988604'), ('name', 'mythological Greek character')]
[('obj', 'http://www.wikidata.org/entity/Q95074'), ('name', 'fictional character')]
3


In [33]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q178885"
og_name = "deity"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 1_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_4.json
JSON object updated


`Q178885`-> deity

## Task 7

In [34]:
## try to find correlation between Poseidon and Ancient Greece
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q41127 ?prop ?x.
    ?prop sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1049'), ('name', 'worshipped by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1299'), ('name', 'depicted by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1441'), ('name', 'present in work')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1871'), ('name', 'CERL Thesaurus ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender')]
[('prop', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P22'

In [35]:
## try to find correlation between Poseidon and Ancient Greece
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q41127 wdt:P361 ?x.
    ?x sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q101609'), ('name', 'Twelve Olympians')]
1


In [36]:
## try to find correlation between Poseidon and Ancient Greece
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q101609 ?prop ?x.
    ?prop sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P2924'), ('name', 'Great Russian Encyclopedia Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P4212'), ('name', 'PACTOLS thesaurus ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P443'), ('name', 'pronunciation audio')]
[('prop', 'http://www.wikidata.org/prop/direct/P5063'), ('name', 'Interlingual Index ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P527'

In [38]:
## list the twelve olympians
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q101609 wdt:P527 ?x.
    ?x sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q41127'), ('name', 'Poseidon')]
[('x', 'http://www.wikidata.org/entity/Q34201'), ('name', 'Zeus')]
[('x', 'http://www.wikidata.org/entity/Q35500'), ('name', 'Aphrodite')]
[('x', 'http://www.wikidata.org/entity/Q40730'), ('name', 'Demeter')]
[('x', 'http://www.wikidata.org/entity/Q40901'), ('name', 'Ares')]
[('x', 'http://www.wikidata.org/entity/Q41484'), ('name', 'Hermes')]
[('x', 'http://www.wikidata.org/entity/Q37340'), ('name', 'Apollo')]
[('x', 'http://www.wikidata.org/entity/Q37122'), ('name', 'Athena')]
[('x', 'http://www.wikidata.org/entity/Q38012'), ('name', 'Hera')]
[('x', 'http://www.wikidata.org/entity/Q39503'), ('name', 'Artemis')]
[('x', 'http://www.wikidata.org/entity/Q44384'), ('name', 'Hephaestus')]
[('x', 'http://www.wikidata.org/entity/Q41419'), ('name', 'Hestia')]
12


In [39]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 1_4
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow1_4.json
JSON object updated


## Task 8

In [40]:
# list subclasses of deity
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    ?obj wdt:P279 wd:Q178885.
    ?obj sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11688446'), ('name', 'Roman deity')]
[('obj', 'http://www.wikidata.org/entity/Q13405593'), ('name', 'nature deity')]
[('obj', 'http://www.wikidata.org/entity/Q190'), ('name', 'God')]
[('obj', 'http://www.wikidata.org/entity/Q979507'), ('name', 'Hindu deity')]
[('obj', 'http://www.wikidata.org/entity/Q205985'), ('name', 'goddess')]
[('obj', 'http://www.wikidata.org/entity/Q22989102'), ('name', 'Greek deity')]
[('obj', 'http://www.wikidata.org/entity/Q24284226'), ('name', 'agricultural deity')]
[('obj', 'http://www.wikidata.org/entity/Q32066628'), ('name', 'Aztec deity')]
[('obj', 'http://www.wikidata.org/entity/Q7805266'), ('name', 'time and fate deity')]
[('obj', 'http://www.wikidata.org/entity/Q2072049'), ('name', 'yidam')]
[('obj', 'http://www.wikidata.org/entity/Q1670614'), ('name', 'love deity')]
[('obj', 'http://www.wikidata.org/entity/Q7813363'), ('name', 'Toilet god')]
[('obj', 'http://www.wikidata.org/entity/Q1589492'), ('name',

In [41]:
# properties of Greek deity
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q22989102 ?prop ?obj.
    ?prop sc:name ?name.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P2596'), ('name', 'culture')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
6


In [42]:
# find for the ancient civilizations their deities
queryString = """
SELECT DISTINCT ?obj ?name ?dei ?dName
WHERE { 
    ?obj wdt:P31 wd:Q28171280.
    ?dei wdt:P279 wd:Q178885;
         wdt:P2596 ?obj.
    ?obj sc:name ?name.
    ?dei sc:name ?dName.
} 
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt'), ('dei', 'http://www.wikidata.org/entity/Q146083'), ('dName', 'Ancient Egyptian deity')]
[('obj', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece'), ('dei', 'http://www.wikidata.org/entity/Q22989102'), ('dName', 'Greek deity')]
2


In [108]:

queryString = """
SELECT DISTINCT ?obj ?name  COUNT(DISTINCT ?div) as ?deity
WHERE { 
    ?obj wdt:P31 wd:Q28171280.
    ?dei wdt:P279 wd:Q178885;
         wdt:P2596 ?obj.
    ?div wdt:P31 ?dei.
    ?obj sc:name ?name.
} 
GROUP BY  ?obj ?name
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11772'), ('name', 'Ancient Greece'), ('deity', '252')]
[('obj', 'http://www.wikidata.org/entity/Q11768'), ('name', 'Ancient Egypt'), ('deity', '281')]
2
